In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from shutil import copyfile
import numpy as np
import os

In [ ]:
mypath= '/content/drive/My Drive/Pengpol/data_pengpol/'
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(mypath):        #os.walk: baca mypath dan masukin nama folder, cth: Lesi Jinak, dll, serta isi dari setiap file
    for name in files:
        full_path.append(os.path.join(path, name))  #full_path.append: menyimpan yg dibawa os.walk agar path otomatis diakses
        tag.append(path.split('/')[-1])             #tag.append: mundur 1 directory untuk cek Lesi Jinak, dll (1 direc di atas)
        file_name.append(name)                      #file_name.append(name): isi file_name dgn name: ....400x.jpg

In [ ]:
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})  #pd.DataFrame: buat tabel dgn label path, file_name, tag
df.groupby(['tag']).size()                                             #dr.groupby: Hitung size data2 dari tiap tag
#df: nama tabel dgn label path, file_name, tag

tag
Carcinoma      98
Lesi Jinak     97
Limfoma       235
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#variabel yang digunakan pada pemisahan data ini
X= df['path']           #
y= df['tag']            #

In [ ]:
# split dataset awal menjadi data train dan test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=300)

In [ ]:
# kemudian data test dibagi menjadi 2 sehingga menjadi data test dan data validation.
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.5, random_state=100)

In [ ]:
# menyatukan kedalam masing-masing dataframe

df_tr = pd.DataFrame({'path':X_train
              ,'tag':y_train
             ,'set':'train'})

df_te = pd.DataFrame({'path':X_test
              ,'tag':y_test
             ,'set':'test'})

df_val = pd.DataFrame({'path':X_val
              ,'tag':y_val
             ,'set':'validation'})

In [ ]:
print('train size', len(df_tr))
print('val size', len(df_te))
print('test size', len(df_val))

train size 344
val size 43
test size 43


In [ ]:
# melihat proporsi pada masing masing set apakah sudah ok atau masih ada yang ingin diubah
df_all = df_tr.append([df_te,df_val]).reset_index(drop=1)\

print('===================================================== \n')
print(df_all.groupby(['set','tag']).size(),'\n')

print('===================================================== \n')

#cek sample datanya
df_all.sample(3)


set         tag       
test        Carcinoma      11
            Lesi Jinak     13
            Limfoma        19
train       Carcinoma      75
            Lesi Jinak     74
            Limfoma       195
validation  Carcinoma      12
            Lesi Jinak     10
            Limfoma        21
dtype: int64 




,path,tag,set
161,/content/drive/My Drive/Pengpol/data_pengpol/L...,Limfoma,train
307,/content/drive/My Drive/Pengpol/data_pengpol/C...,Carcinoma,train
341,/content/drive/My Drive/Pengpol/data_pengpol/L...,Limfoma,train


In [ ]:
import shutil
from tqdm.notebook import tqdm as tq

In [ ]:
datasource_path = "/content/drive/My Drive/Pengpol/data_pengpol/"
dataset_path = "/content/drive/My Drive/Pengpol/dataset/"

In [ ]:
for index, row in tq(df_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

In [ ]:
# Parameter input untuk network
dim = (150, 150)
channel = (3, )
input_shape = dim + channel

# Batch size
batch_size = 16

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)  #ImageDataGenerator: fungsi utk data augmentation
val_datagen = ImageDataGenerator(rescale=1. / 255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/Pengpol/dataset/train',
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
val_generator = val_datagen.flow_from_directory('/content/drive/My Drive/Pengpol/dataset/validation',
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/Pengpol/dataset/test',
                                                   target_size=input_shape,
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=True)


Found 344 images belonging to 3 classes.
Found 43 images belonging to 3 classes.
Found 43 images belonging to 3 classes.


In [ ]:
def tf_data_generator(generator, input_shape):
    num_class = generator.num_classes
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None
                        , input_shape[0]
                        , input_shape[1]
                        , input_shape[2]]
                       ,[None, num_class])
    )
    return tf_generator
train_data = tf_data_generator(train_generator, input_shape)
test_data = tf_data_generator(test_generator, input_shape)
val_data = tf_data_generator(val_generator, input_shape)

# MobileNet